In [4]:
!pip install pymysql sqlalchemy

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 162.5 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 245.8 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 221.2 kB/s eta 0:00:00


In [22]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


········


#5. Check the number of rentals for each customer for May.

In [24]:
query = '''select customer_id, count(rental_id) as nb_of_rentals from rentals_may
group by customer_id
order by customer_id asc'''

df1 = pd.read_sql_query(query, engine)
df1

,customer_id,nb_of_rentals
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [25]:
query2 = '''select customer_id, count(rental_id) as nb_of_rentals from rentals_june
group by customer_id
order by customer_id asc'''

df2 = pd.read_sql_query(query2, engine)
df2

,customer_id,nb_of_rentals
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


#6.Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

In [50]:
merged_df = pd.merge(df1, df2, on='customer_id',how='outer', suffixes=('_May', '_June'))
merged_df

,customer_id,nb_of_rentals_May,nb_of_rentals_June
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,5,3.0,5.0
4,6,3.0,4.0
...,...,...,...
593,583,NaN,6.0
594,585,NaN,4.0
595,591,NaN,3.0
596,592,NaN,5.0


In [66]:
merged_df = merged_df.fillna(0)
merged_df

,customer_id,nb_of_rentals_May,nb_of_rentals_June
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,5,3.0,5.0
4,6,3.0,4.0
...,...,...,...
593,583,0.0,6.0
594,585,0.0,4.0
595,591,0.0,3.0
596,592,0.0,5.0


In [61]:
def june_vs_may(merged_df, customer_id):
    
    row = merged_df[merged_df['customer_id'] == customer_id]
    
    nb_of_rentals_June = row['nb_of_rentals_June'].values[0]
    nb_of_rentals_May = row['nb_of_rentals_May'].values[0]
    
    if nb_of_rentals_June > nb_of_rentals_May :
        return f" This customer rented {nb_of_rentals_June-nb_of_rentals_May} more movies in June than in May"
    elif nb_of_rentals_June <nb_of_rentals_May :
        return f"This customer rented {nb_of_rentals_June-nb_of_rentals_May} less movies in June than in May"
    else:
        return "This customer rented as many movies in June as in May."
    


In [67]:
june_vs_may(merged_df,598)

' This customer rented 1.0 more movies in June than in May'